In [3]:
from pathlib import Path
import pandas as pd

In [ ]:
# Directorios
base_dir = Path().resolve()

# Rutas de CSVs
path_csv_hist = base_dir.parent / "franco" / "API-connect" / "posts_con_sentimiento_historia.csv"
path_csv_agrupado = base_dir.parent / "franco" / "API-connect" / "analisis_sentimiento_diario.csv"

NameError: name '__file__' is not defined

In [ ]:
# Lectura CSVs
df_hist = pd.read_csv(path_csv_hist)
df_diario = pd.read_csv(path_csv_agrupado)
df_diario = df_diario.rename(columns={"date": "fecha"})

In [ ]:
df_diario.head()

In [ ]:
# Transformaciones
df_diario['fecha'] = pd.to_datetime(df_diario['fecha'], utc=True, errors='coerce')
df_hist['created_at'] = pd.to_datetime(df['created_at'], utc=True, errors='coerce')


# recorto horarios
df_diario['fecha'] = df_diario['fecha'].dt.date
df['fecha'] = df['created_at'].dt.date

# 1. Asegurarte de que created_at sea datetime
df = df_hist.copy()

